# GETTIN
## Planejamento de Equipamentos Escolares
## Método: Capacitaded Facility Location
### Versão: 0.5
#### Fillipe O Feitosa <fillipefeitosa@ua.pt>

---


\begin{split}& \text{minimize} \quad    & \sum_{j=1}^m f_j x_j +\sum_{i=1}^n \sum_{j=1}^m c_{ij} y_{ij} &     \\
& \text{subject to:} \quad & \sum_{j=1}^m y_{ij} =d_i  &  \mbox{ for }  i=1,\cdots,n \\
&    & \sum_{i=1}^n y_{ij} \leq M_j x_j & \mbox{ for } j=1,\cdots,m  \\
&    & y_{ij} \leq d_i x_j              & \mbox{ for } i=1,\cdots,n; j=1,\cdots,m \\
&    & y_{ij} \geq 0                    & \mbox{ for } i=1,\cdots,n; j=1,\cdots,m \\
&    & x_j \in \{ 0,1 \}                & \mbox{ for } j=1,\cdots,m\end{split}

#### Modeling and Creating Decision Variables

In [1]:
# Import Libraries
from gurobi import *
import math
import matplotlib.pyplot as plt

def distance(a,b):
    dx = a[0] - b[0]
    dy = a[1] - b[1]
    return math.sqrt(dx*dx + dy*dy)

# Data
# Problem data


# --------- Capacitaded - Dados de TESTE -----------#
# I, d
# Retirar os Centroides de Cada Subsecao - OK
# Adicionar a Demanda de cada Subsecao atravez da Planinlha
subSecao, subDemanda = multidict({1:80, 2:270, 3:250, 4:160, 5:180})
# J, M, f
# Cada possível Escola é um centroid também - OK
# Adicionar um valor fixo para cada Escola  - custo Anual
# Adicionar um valor fixo para a capacidade da Escola
escola, capacidadeEscola, custoAnual = multidict({1:[500,1000], 2:[500,4000], 3:[500,8500]})
# c
# Fazer uma Matriz (como no modelo antigo), usando as distancias
custoTransporte = {(1,1):4,  (1,2):6,  (1,3):9,
     (2,1):5,  (2,2):4,  (2,3):7,
     (3,1):6,  (3,2):3,  (3,3):4,
     (4,1):8,  (4,2):5,  (4,3):3,
     (5,1):10, (5,2):8,  (5,3):4,
     }


# import geopandas as gpd

# vagos = gpd.read_file('./data_gettin/vagos.geojson');
# centroids = vagos.centroid

# iteratorHandler = centroids.size
# centroidVector = []
# for centroid in centroids:
#     obj = [centroid.xy[0][0], centroid.xy[1][0]]
#     centroidVector.append(obj)

# subSections = centroidVector
# schools = centroidVector
# # @charge: custo de construcao por escola (fixo em 500 mil euros)
# charge = 5000000

numSchools = len(escola)
numSubSections = len(subSecao)

# Creting Guroby Model
m = Model()

# Decision Variables
x = {}
y = {}
d = {} # Distance Matrix
# @alpha: 0.29 de custo por Km  por (365 dias * 5 anos) 
# alpha = 529.25

# creating binary variable for every school
for j in escola:
    x[j] = m.addVar(vtype=GRB.BINARY, name="escola(%d)" % j)

# creating continuous variable for subsections to check suply fractions
for i in subSecao:
    for j in escola:
        # Fractions of Subsection Suply
        y[(i,j)] = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Fração da Sub[%d], escola[%d]" % (i,j))
        # Distances between Subsections and Schools
        # d[(i,j)] = distance(subSecao[i], escola[j])

m.update()

In [2]:
custoTransporte[1,1]
subDemanda[1]
soma = 0
for r in subSecao:
    soma += subDemanda[r]
soma

940

## Adding Constraints

In [3]:
# Constraint for Every Student on School
# for i in range(numSubSections):
#     m.addConstr(quicksum(y[(i,j)] for j in range(numSchools)) == 1)

# Constraint to repect demands on Every Subsection
for i in subSecao:
    m.addConstr(quicksum(y[(i,j)] for j in escola) == subDemanda[i])
    
# Constraint to restrain school capacity
for j in capacidadeEscola:
    m.addConstr(quicksum(y[(i,j)] for i in subSecao) <= capacidadeEscola[j] * x[j])
    
for (i,j) in y:
    m.addConstr(y[i,j] <= subDemanda[i]*x[j], "Strong(%s,%s)"%(i,j))

# Fraction of suply must be lower or equal than the BINARY x[j] available 
# for i in range(numSubSections):
#     for j in range(numSchools):
#         m.addConstr(y[(i,j)] <= x[j])

#### Objetive


In [4]:
# Setting objective

m.setObjective(
    quicksum(custoAnual[j]*x[j] for j in escola) + 
    quicksum(custoTransporte[i,j]*y[i,j] for i in subSecao for j in escola), GRB.MINIMIZE )


m.optimize()

Optimize a model with 23 rows, 18 columns and 63 nonzeros
Variable types: 15 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [3e+00, 8e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+01, 3e+02]
Presolve time: 0.00s
Presolved: 23 rows, 18 columns, 63 nonzeros
Variable types: 15 continuous, 3 integer (3 binary)

Root relaxation: objective 9.529200e+03, 11 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 9529.20000    0    1          - 9529.20000      -     -    0s
H    0     0                    9840.0000000 9529.20000  3.16%     -    0s
     0     0     cutoff    0      9840.00000 9840.00000  0.00%     -    0s

Explored 1 nodes (11 simplex iterations) in 0.03 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 9840 

Optimal solution found (tolerance 1

In [5]:
m

<gurobi.Model MIP instance Unnamed: 23 constrs, 18 vars, Parameter changes: LogFile=gurobi.log, CSIdleTimeout=1800>

In [6]:
print('Obj: %g' % m.objVal)

Obj: 9840


In [7]:
for v in m.getVars():
    if(v.x != 0):
       print('%s   %g' % (v.varName, v.x))

escola(1)   1
escola(2)   1
Fração da Sub[1], escola[1]   80
Fração da Sub[2], escola[1]   270
Fração da Sub[3], escola[2]   250
Fração da Sub[4], escola[2]   160
Fração da Sub[5], escola[1]   90
Fração da Sub[5], escola[2]   90


In [8]:
m = None

In [9]:
disposeDefaultEnv()

# 